In [1]:
import pandas as pd
import numpy as np

def topsis_analysis(df, criteria_columns, weights, impacts):
    """
    Perform TOPSIS analysis on the dataset
    
    Parameters:
    - df: Input DataFrame
    - criteria_columns: List of columns to use for TOPSIS
    - weights: List of weights for each criterion
    - impacts: List of impact directions ('+' for maximize, '-' for minimize)
    
    Returns:
    - DataFrame with TOPSIS scores and ranks
    """
    # Create a copy of the dataframe
    data = df.copy()
    
    # Normalize the decision matrix
    normalized_matrix = data[criteria_columns].copy()
    for col in criteria_columns:
        # Normalize using vector normalization
        normalized_matrix[col] = normalized_matrix[col] / np.sqrt((normalized_matrix[col]**2).sum())
    
    # Apply weights
    weighted_matrix = normalized_matrix.copy()
    for i, col in enumerate(criteria_columns):
        weighted_matrix[col] *= weights[i]
    
    # Determine ideal best and worst solutions
    ideal_best = []
    ideal_worst = []
    for i, col in enumerate(criteria_columns):
        if impacts[i] == '+':
            ideal_best.append(weighted_matrix[col].max())
            ideal_worst.append(weighted_matrix[col].min())
        else:
            ideal_best.append(weighted_matrix[col].min())
            ideal_worst.append(weighted_matrix[col].max())

    # Calculate distances to ideal best and worst solutions
    distance_best = np.sqrt(((weighted_matrix[criteria_columns] - ideal_best)**2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_matrix[criteria_columns] - ideal_worst)**2).sum(axis=1))
    
    # Calculate TOPSIS score
    data['Topsis_Score'] = distance_worst / (distance_best + distance_worst)
    
    # Rank the alternatives
    data['Rank'] = data['Topsis_Score'].rank(method='dense', ascending=False)
    
    return data

for YEAR in range(2023, 2039):
    # Read the dataset
    df = pd.read_csv(f'data/topsis_input_{YEAR}.csv')
    df['Total_Cost'] = df['insurance_cost'] + df['maintenance_cost'] + df['fuel_costs']

    # Get unique combinations of Size and Distance_x
    combinations = df.groupby(['Size', 'Distance_x'])

    # Store results
    results = []

    # Perform TOPSIS for each combination
    for (size, distance), group in combinations:
        # Define criteria columns
        criteria_columns = ['carbon_emissions','Total_Cost']
        
        # Define weights (equal weights in this case)
        weights = [0.5, 0.5]
        
        # Define impact directions (both to be minimized)
        impacts = ['-', '-']
        
        # Perform TOPSIS
        result = topsis_analysis(group, criteria_columns, weights, impacts)
        
        # Add size and distance info to results
        result['Size'] = size
        result['Distance'] = distance
        
        results.append(result)

    # Combine all results
    final_results = pd.concat(results)

    # Save to CSV
    final_results.to_csv(f'topsis_results_{YEAR}.csv', index=False)

    print(f"TOPSIS analysis for {YEAR} completed. Results saved to topsis_results_{YEAR}.csv")

TOPSIS analysis for 2023 completed. Results saved to topsis_results_2023.csv
TOPSIS analysis for 2024 completed. Results saved to topsis_results_2024.csv
TOPSIS analysis for 2025 completed. Results saved to topsis_results_2025.csv
TOPSIS analysis for 2026 completed. Results saved to topsis_results_2026.csv
TOPSIS analysis for 2027 completed. Results saved to topsis_results_2027.csv
TOPSIS analysis for 2028 completed. Results saved to topsis_results_2028.csv
TOPSIS analysis for 2029 completed. Results saved to topsis_results_2029.csv
TOPSIS analysis for 2030 completed. Results saved to topsis_results_2030.csv
TOPSIS analysis for 2031 completed. Results saved to topsis_results_2031.csv
TOPSIS analysis for 2032 completed. Results saved to topsis_results_2032.csv
TOPSIS analysis for 2033 completed. Results saved to topsis_results_2033.csv
TOPSIS analysis for 2034 completed. Results saved to topsis_results_2034.csv
TOPSIS analysis for 2035 completed. Results saved to topsis_results_2035.csv

In [21]:
final_results

,Unnamed: 0,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),...,Operating Year,Fuel,carbon_emissions,insurance_cost,maintenance_cost,fuel_costs,Total_Cost,Topsis_Score,Rank,Distance
0,0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,...,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,28690.316221,0.918591,1.0,D1
1,1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,...,2023,B20,69347.975963,4250.0,850.00,27771.219051,32871.219051,0.179776,5.0,D1
2,2,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,...,2023,HVO,11092.291306,4250.0,850.00,41271.256214,46371.256214,0.669280,3.0,D1
3,3,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,...,2023,LNG,41971.148530,5000.0,1000.00,17077.879517,23077.879517,0.476128,4.0,D1
4,4,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,...,2023,BioLNG,6375.265894,5000.0,1000.00,18325.652865,24325.652865,0.912129,2.0,D1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,46,LNG_S4_2023,2023,S4,D3,118899,LNG_S4_2023,LNG,193454,118000,...,2023,BioLNG,7416.792843,9672.7,1934.54,21319.514083,32926.754083,0.980303,1.0,D3
47,47,Diesel_S4_2023,2023,S4,D4,1809,Diesel_S4_2023,Diesel,151272,118000,...,2023,B20,79078.903088,7563.6,1512.72,31668.084172,40744.404172,0.194361,4.0,D4
48,48,Diesel_S4_2023,2023,S4,D4,1809,Diesel_S4_2023,Diesel,151272,118000,...,2023,HVO,12648.764684,7563.6,1512.72,47062.450276,56138.770276,0.700796,2.0,D4
49,49,LNG_S4_2023,2023,S4,D4,1809,LNG_S4_2023,LNG,193454,118000,...,2023,LNG,49073.243635,9672.7,1934.54,19967.691418,31574.931418,0.495760,3.0,D4
